# Reduced Density Matrices in Tequila

This notebook serves as a tutorial to the computation and usage of the one- and two-particle reduced density matrices.

In [5]:
import tequila as tq
import numpy

## The 1- and 2-RDM

First, look at the definition of the reduced density matrices for some state $ |\psi\rangle = U |0\rangle^{\otimes n}$ on an $n$-qubit system ($\Leftrightarrow$ $n$ spin-orbitals in Jordan-Wigner encoding).
### todo: N particles $\leftrightarrow$ n spin-orbitals

$$ \gamma^p_q \equiv \langle \psi | a^p a_q | \psi\rangle$$

$$ \gamma^{pq}_{rs} \equiv \langle \psi | a^p a^q a_s a_r | \psi\rangle$$ (we mainly use the standard physics ordering for the second-quantized operators)

using the standard fermionic creation and annihilation operators $$ a^p = a_p^\dagger $$ and $$a_p$$.

The well-known anticommutation relations yield a series of symmetry properties for the reduced density matrices

Further, we can find the spin-free correspondents $\Gamma^P_Q$ and $\Gamma^{PQ}_{RS}$

Here, some small explanations for symmetries...

In [3]:
# As an example, let's use the Helium atom in a minimal basis
mol = tq.chemistry.Molecule(geometry='He 0.0 0.0 0.0', basis_set='6-31g')

# We need to set up a quantum circuit to solve this problem.
# This can be done either using the make_uccsd-method or by a hand-written circuit
U = tq.gates.X(0, 1) # and so on

You can initialize a tequila `QubitHamiltonian` from a molecule with `make_hamiltonian`.
The standard transformation is the `jordan-wigner` transformation.  
You can use other transformations by initializing the molecule with the `transformation` keyword.

You can also use different encodings for the computation, though note, that in order to compare with e.g. a psi4 rdm or
in general a usual RDM, need to either use Jordan-Wigner (occupation number encoding) or do some post-treatment.


We can compute the following:

Energy $E = \langle H \rangle = \langle h^q_p a^p_q + h^{rs}_{pq}a^{pq}_{rs}\rangle = h^q_p \langle a^p_q \rangle + h^{rs}_{pq}\langle a^{pq}_{rs}\rangle = h^q_p \gamma^p_q + h^{rs}_{pq} \gamma^{pq}_{rs}$
toc check consistency.
Also, we can use the fact, that
$\mathrm{tr} (\mathbf{\gamma}_1) = N $ and $ \mathrm{tr} (\mathbf{\gamma}_2) = N(N-1)/2$




